Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [1]:
import selenium   
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Website address
url = 'https://www.amazon.in/'
# variable defined
prod_name = []
prod_desc = []
prod_price = []
prod_return = []
prod_del = []
prod_avail = []
prod_url = []

# function for scraping data using BeautifulSoup

def scraping_web_page(driver):
    soup = BeautifulSoup(driver.page_source,'html.parser')
    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    for p in products:
        try:
            brand_name = p.find('span', class_='a-size-base-plus a-color-base a-text-normal').text.split()
        except:
            brand_name = "-"
        try:
            brand_desc = p.find('span', class_='a-size-base-plus a-color-base a-text-normal').text.strip()
        except:
            brand_desc = '-'
        try:
            brand_price = p.find('span', class_='a-price-whole').text.strip()
        except:
            brand_price = '-'
        try:
            brand_return = p.find('div', {'aria-label': 'Return this item'}).text.strip()
        except:
            brand_return = '-'
        try:
            brand_del = p.find('span', class_='a-text-bold').text.strip()
        except:
            brand_del = '-'
        try:
            brand_avail = p.find('span', class_='a-size-base').text.strip()
        except:
            brand_avail = '-'
        try:
            url = p.find('a', class_='a-link-normal') ['href']
                    
        except:
             url = '-'
        # adding data to the list 
        prod_name.append(brand_name[0])
        prod_desc.append(brand_desc)
        prod_price.append(brand_price)
        prod_return.append(brand_return)
        prod_del.append(brand_del)
        prod_avail.append(brand_avail)
        prod_url.append(url)
    
    return prod_name,prod_desc,prod_price, prod_return, prod_del, prod_avail, prod_url

    
#user input
prodname = input("Enter product name: ").upper()

# create a webdriver instance using chrome
driver = webdriver.Chrome()

#open web page
driver.get(url)

try:
    # wait till the element is not located
    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.ID,'twotabsearchtextbox')))
    #Find the search field and enter the product name entered by user
    search = driver.find_element(By.ID,'twotabsearchtextbox')
    search.send_keys(prodname)
    search.submit()

except TimeoutException:
    driver.get(url)
    # wait till the element is not located
    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.ID,'twotabsearchtextbox')))
    #Find the search field and enter the product name entered by user
    search = driver.find_element(By.ID,'twotabsearchtextbox')
    search.send_keys(prodname)
    search.submit()   
    
# Initialize lists to store product details
all_prod_name = []
all_prod_desc = []
all_prod_price = []
all_prod_return = []
all_prod_del = []
all_prod_avail = []
all_prod_url = []
    
# Scrape product details from the first 3 pages

for page in range(1, 4):
    print(f'Scraping page {page}...')
    time.sleep(2)  # Add a delay to allow the page to load
    
    # Scrape details from the current page
    prod_name,prod_desc, prod_price, prod_return, prod_del, prod_avail, prod_url = scraping_web_page(driver)
    
    all_prod_name.extend(prod_name)
    all_prod_desc.extend(prod_desc)
    all_prod_price.extend(prod_price)
    all_prod_return.extend(prod_return)
    all_prod_del.extend(prod_del)
    all_prod_avail.extend(prod_avail)
    all_prod_url.extend(prod_url)
          
    # Navigate to the next page if available
    try:
        next_page = driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[69]/div/div/span/a[3]')
        next_page.click()
    except NoSuchElementException:
        print(f"continue {page}...")
    
# data frame defined            
df = pd.DataFrame({"Name":all_prod_name,
                   "Product Description":all_prod_desc,
                   "Product Price": all_prod_price,
                   "Product Return":all_prod_return,
                   "Product Delivery":all_prod_del,
                   "Product Avail":all_prod_avail,
                  "Product URL":all_prod_url})

df.to_csv('C:\\Users\\vrosa\\Desktop\\DSML\\Internship\\products.csv') # writing the data to CSV file
print('Data saved to products.csv')
driver.quit() # closing the browser
df

Enter product name: guitar
Scraping page 1...
continue 1...
Scraping page 2...
continue 2...
Scraping page 3...
continue 3...
Data saved to products.csv


,Name,Product Description,Product Price,Product Return,Product Delivery,Product Avail,Product URL
0,Washburn,"Washburn 6 String Acoustic Guitar, Right (DFED)","17,490",-,"Tuesday, 20 February",1,/sspa/click?ie=UTF8&spc=MToxNzQ0MzI5NTIwODkwMD...
1,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","5,499",-,"Tuesday, 20 February","1,164",/sspa/click?ie=UTF8&spc=MToxNzQ0MzI5NTIwODkwMD...
2,Kadence,Kadence Frontier guitar with Online Guitar lea...,"4,499",-,"Tuesday, 20 February","3,101",/sspa/click?ie=UTF8&spc=MToxNzQ0MzI5NTIwODkwMD...
3,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"9,999",-,"Tuesday, 20 February",653,/sspa/click?ie=UTF8&spc=MToxNzQ0MzI5NTIwODkwMD...
4,Intern,"Intern INT-38C Right hand Acoustic Guitar Kit,...","1,999",-,"Tuesday, 20 February","14,217",/gp/bestsellers/musical-instruments/4654375031...
...,...,...,...,...,...,...,...
355,Intern,Intern INT-38C-SB-G Linden Wood Cutaway Right ...,"2,099",-,"Tuesday, 20 February","1,772",/Intern-INT-38C-SB-G-Guitar-Kit-Sunburst/dp/B0...
356,Right,Right Search ®Musical Guitar and Drum Combo Se...,"1,099",-,"Tuesday, 20 February",M.R.P:,/sspa/click?ie=UTF8&spc=MToxNzQ0MzI5NTIwODkwMD...
357,Right,Right Search ® Musical Guitar Toy_g2,299,-,"Tuesday, 20 February",M.R.P:,/sspa/click?ie=UTF8&spc=MToxNzQ0MzI5NTIwODkwMD...
358,Guitar,"Guitar Tailpiece, Guitars Bridge Tailpiece All...","1,399.",-,"Monday, 19 February",M.R.P:,/sspa/click?ie=UTF8&spc=MToxNzQ0MzI5NTIwODkwMD...


Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [45]:
import selenium   
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import warnings
warnings.filterwarnings('ignore') 

#search list with serch items
prodname = ['fruits','cars','Machine Learning','Guitar','Cakes'] 

url = 'https://images.google.com/' # website address

driver = webdriver.Chrome() # browser to be used

#open web page
driver.get(url)

# wait till the element is not located
WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CLASS_NAME,"gLFyf")))

for p in range (len(prodname)):
    try:                                                        #try and except block for exception handeling
        search = driver.find_element(By.CLASS_NAME,'gLFyf')
        search.clear()
        search.send_keys(prodname[p])
        search.submit()
        time.sleep(2)
        current_url = driver.current_url
        time.sleep(5)
                
    except (NoSuchElementException,StaleElementReferenceException):
        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//input[@class="og3lId"]')))
        #Find the search field and enter the product name entered by user
        search = driver.find_element(By.XPATH,'//input[@class="og3lId"]')
        search.clear()
        search.send_keys(prodname[p])
        search.submit()
        time.sleep(2)
        current_url = driver.current_url
        time.sleep(3)
        
    driver.get(current_url) # getting the current url
    
    for _ in range(10):
        driver.execute_script("window.scrollBy(0,500)") #loading the item 
        
    images= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
    img_urls = []
    
    for image in images: #getting the image url
        source = image.get_attribute("src")
        if source is not None:
            if(source[0:4]=='http'):
                img_urls.append(source)
                
    for x in range(0,10):
        response = requests.get(img_urls[x]) # downloading the images 
        with open(r'C:\Users\vrosa\Desktop\DSML\Internship\img\\'+ prodname[p] + str(x) + '.jpg',"wb") as f: # writing images
            f.write(response.content) 
print("Image downloaded") 

driver.quit() #closing browser
        

Image downloaded


Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [42]:
import selenium   
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Variable defined
brand_name =[]
smartphone = []
color = []
ram = []
storage = []
iname =[]
camera = []
display = []
battery = []
price = []
purl = []

# function for getting data from the table
def table_data(driver):
    cell_value=[]
    # Locate the table element (replace 'table_id' with the actual ID or use other locators)
    table = driver.find_element(By.XPATH,'//table[@class="_14cfVK"]')
    # Retrieve all rows within the table
    rows = table.find_elements(By.TAG_NAME,'tr')
    # Iterate through rows
    for row in rows:
        # Retrieve all cells within the current row
        cells = row.find_elements(By.TAG_NAME,'td')
        # Extract data from each cell and print it
        for cell in cells:
            cell_value.append(cell.text)  # Extract text content of the cell

    return cell_value

    

url = 'https://www.flipkart.com/' # wibsite address
smartitem = input(f"Enter Smartphone (e.g.: Oneplus Nord, pixel 4A, etc.) : ") #taking input from the user
driver = webdriver.Chrome() # defining browser
#open web page
driver.get(url)

# path for search field and submitting the data to search
search = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div/div/div/div[1]/div/div[1]/div/div[1]/div[1]/header/div[1]/div[2]/form/div/div/input')
search.send_keys(smartitem)
search.submit()
time.sleep(3)

# fetching individual phone weblink
item_name = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for item in item_name:
    iname = item.get_attribute("href")
    all_iname.append(iname)

# fetching individual phone details
for l in all_iname: 
    driver.get(l)
    time.sleep(3)
    iname = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]').text.split()
    sphone = table_data(driver)
    pstorage = driver.find_element(By.XPATH,'//div[@class="_2418kt"]/ul/li').text
    pdisplay = driver.find_element(By.XPATH,'//div[@class="_2418kt"]/ul/li[2]').text
    pcamera = driver.find_element(By.XPATH,'//div[@class="_2418kt"]/ul/li[3]').text
    pbattery = driver.find_element(By.XPATH,'//div[@class="_2418kt"]/ul/li[4]').text
    pprice = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]').text
    
    # try and except block for storing data in list.if the data is not available hyphen(-) will be entered
    try:
        brand_name.append(iname[0])
    except:
        brand_name.append("-")
    try:
        smartphone.append(sphone[5])
    except:
        smartphone.append("-")
    try:
        color.append(sphone[6])
    except:
        color.append("-")
    try:
        storage.append(pstorage)
    except:
        storage.append("-")
    try:
        display.append(pdispaly)
    except:
        display.append("-")
    try:
        camera.append(pcamera)
    except:
        camera.append("-")
    try:
        battery.append(pbattery)
    except:
        battery.append("-")
    try:
        price.append(pprice)
    except:
        price.append("-")
    try:
        purl.append(driver.current_url)
    except:
        purl.append("-")       
        
        
  # defining data frame      
                      
df1 = pd.DataFrame({"Brand Name " : brand_name,
                   "Smartphone Name":smartphone,
                   "Phone Color":color,
                   "Storage(RAM | ROM)":storage,
                   "Phone Dispaly":display,
                   "Phone Camera":camera,
                   "Phone Battery":battery,
                   "Phone Price":price,
                   "URL":purl
                 })

df1.to_csv('C:\\Users\\vrosa\\Desktop\\DSML\\Internship\\mobiles.csv',index=False) # writing data in csv file
print('Data saved to mobiles.csv')
driver.quit() # closing browser
df1

Enter Smartphone (e.g.: Oneplus Nord, pixel 4A, etc.) : pixel
Data saved to mobiles.csv


,Brand Name,Smartphone Name,Phone Color,Storage(RAM | ROM),Phone Dispaly,Phone Camera,Phone Battery,Phone Price,URL
0,Google,Pixel 7a,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,64MP (OIS) + 13MP | 13MP Front Camera,4300 mAh Battery,"₹40,999",https://www.flipkart.com/google-pixel-7a-coral...
1,Google,Pixel 7a,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,64MP (OIS) + 13MP | 13MP Front Camera,4300 mAh Battery,"₹40,999",https://www.flipkart.com/google-pixel-7a-charc...
2,Google,Pixel 7a,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,64MP (OIS) + 13MP | 13MP Front Camera,4300 mAh Battery,"₹40,999",https://www.flipkart.com/google-pixel-7a-snow-...
3,Google,Pixel 7a,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,64MP (OIS) + 13MP | 13MP Front Camera,4300 mAh Battery,"₹40,999",https://www.flipkart.com/google-pixel-7a-sea-1...
4,Google,Pixel 8,Color,8 GB RAM | 256 GB ROM,17.12 cm (6.74 inch) HD Display,50MP + 12MP | 10.5MP Front Camera,4575 mAh Battery,"₹82,999",https://www.flipkart.com/google-pixel-8-hazel-...
5,Google,Pixel 8,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,50MP + 12MP | 10.5MP Front Camera,4575 mAh Battery,"₹75,999",https://www.flipkart.com/google-pixel-8-rose-1...
6,Google,Pixel 8,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,50MP + 12MP | 10.5MP Front Camera,4575 mAh Battery,"₹75,999",https://www.flipkart.com/google-pixel-8-hazel-...
7,Google,Pixel 8,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,50MP + 12MP | 10.5MP Front Camera,4575 mAh Battery,"₹75,999",https://www.flipkart.com/google-pixel-8-obsidi...
8,Google,Pixel 7,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,50MP + 12MP | 10.8MP Front Camera,4270 mAh Battery,"₹46,999",https://www.flipkart.com/google-pixel-7-lemong...
9,Google,Pixel 7,Color,8 GB RAM | 128 GB ROM,17.12 cm (6.74 inch) HD Display,50MP + 12MP | 10.8MP Front Camera,4270 mAh Battery,"₹46,999",https://www.flipkart.com/google-pixel-7-obsidi...


Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [68]:
import selenium   
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# website addres
url = 'https://www.google.com/maps/'
city_name = input(f"Enter the City name : ") # asking user to input city name
driver = webdriver.Chrome() # choosing browser
#open web page
driver.get(url) # open wedsite

# locate search field and enter input and click the search icon
search = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input')
search.send_keys(city_name)
click_search = driver.find_element(By.XPATH,'//span[@class="google-symbols"]')
click_search.click()
time.sleep(3)

# getting the current website address. Fetching latitude and longitude from url
current_url = driver.current_url
pat = r'@' 
coordinate = current_url.split('/') 
directions = coordinate[6].split(',')
latitude = re.sub(pat,'',directions[0])
longitude = directions[1]

print(f'Latitude {latitude} and Longitude {longitude}')

Enter the City name : Delhi
Latitude 28.6442873 and Longitude 76.7635424


Q6.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
# Need to work on this

Q7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [9]:
import selenium   
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# variable defined
rank = []
mname = []
networth = []
age =[]
country =[]
territory = []
source = []
ranko=[]

# website link, defining browser, searching element to go the desired page
url = 'https://www.forbes.com/'
driver = webdriver.Chrome()
#open web page
driver.get(url)
time.sleep(3)
page = driver.find_element(By.ID,'globalHeaderMenu')
page.click()
pmenu = driver.find_element(By.XPATH,'//div[@class="mpBfVZz3"]')
pmenu.click()

# try and except block for exception handeling
try:
    imenu = driver.find_element(By.XPATH,'//li[@class="TjJgrPSg _2bNo56RE secondary"]/a')
    imenu.click()
except:
    driver.quit()
    driver.get(url)
    time.sleep(3)
    page = driver.find_element(By.ID,'globalHeaderMenu')
    page.click()
    pmenu = driver.find_element(By.XPATH,'//div[@class="mpBfVZz3"]')
    pmenu.click()

time.sleep(3)

# getting current wedaddress
current_url = driver.current_url
driver.get(current_url)

# scraping data from the web portal and storing in list
ranko= driver.find_element(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[2]').text.split('\n')

rank.append(ranko[0])
mname.append(ranko[1])
networth.append(ranko[2])
age.append(ranko[3])
country.append(ranko[4])
territory.append(ranko[5])
source.append(" ")

frank = driver.find_elements(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[@class="TableRow_rowContainer__IC1Tv"]//div[@class="TableRow_row__L-0Km"]/div[1]')
fname = driver.find_elements(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[@class="TableRow_rowContainer__IC1Tv"]//div[@class="TableRow_row__L-0Km"]/div[2]')
fnetworth = driver.find_elements(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[@class="TableRow_rowContainer__IC1Tv"]//div[@class="TableRow_row__L-0Km"]/div[3]')
fage = driver.find_elements(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[@class="TableRow_rowContainer__IC1Tv"]//div[@class="TableRow_row__L-0Km"]/div[4]')
fcountry = driver.find_elements(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[@class="TableRow_rowContainer__IC1Tv"]//div[@class="TableRow_row__L-0Km"]/div[5]')
fterritory = driver.find_elements(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[@class="TableRow_rowContainer__IC1Tv"]//div[@class="TableRow_row__L-0Km"]/div[6]')
fsource = driver.find_elements(By.XPATH,'//div[@class="LeftRail_container__9pEea TableRow_leftRailTable__x2FIq"]/div[@class="TableRow_rowContainer__IC1Tv"]//div[@class="TableRow_row__L-0Km"]/div[7]/div[@class="Table_dataCell__2QCve"]')

for f in frank:
    rank.append(f.text)

for name in fname:
    mname.append(name.text)
    
for net in fnetworth:
    networth.append(net.text)

for a in fage:
    age.append(a.text) 

for c in fcountry:
    country.append(c.text)  
    
for t in fterritory:
    territory.append(t.text)
    
for s in fsource:
    source.append(s.text)

# data frame defined
df= pd.DataFrame({"RANK":rank,
                  "Name":mname,
                  "NetWorth":networth,
                  "Age":age,
                  "Country":country,
                  "Territory":territory,
                  "Source":source})

driver.quit() #closing browser
df

# Due to some reason Source is coming as blank, need to find out, need to investigate further

,RANK,Name,NetWorth,Age,Country,Territory,Source
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",
2,3,Jeff Bezos,$114 B,59,United States,Amazon,
3,4,Larry Ellison,$107 B,78,United States,Oracle,
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,
...,...,...,...,...,...,...,...
195,195,Jin Baofang,$9.6 B,70,China,Solar panels,
196,195,Luo Liguo & family,$9.6 B,67,China,Chemicals,
197,195,Marijke Mars,$9.6 B,58,United States,"Candy, pet food",
198,195,Pamela Mars,$9.6 B,62,United States,"Candy, pet food",


Q8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [12]:
# need to work on this...

import selenium   
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

url = 'https://www.youtube.com/watch?v=ksY3wb4vtlA&list=RDksY3wb4vtlA&start_radio=1'
driver = webdriver.Chrome()
#open web page
driver.get(url)
time.sleep(3)

for _ in range(1000):
    driver.execute_script("window.scrollBy(0,1000)")
time.sleep(3)

# need to work on this..

Q9.Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description. (trouble in searching city)

In [34]:
# need to work on this....

import selenium   
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

url = 'https://www.hostelworld.com/'
driver = webdriver.Chrome()
#driver.maximize_window()
#open web page
driver.get(url)
time.sleep(3)

search = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/input')
search.click()
search.send_keys("London, England")
fetch_data= driver.find_element(By.XPATH,'//div[@class="destination-selection"]/div//ul[@class="select-list"]/li[2]')
fetch_data.click()

WebDriverException: Message: To submit an element, it must be nested inside a form element
